In [1]:
#!/usr/bin/env python
#
# Simple DMRG tutorial.  This code contains a basic implementation of the
# infinite system algorithm
#
# Copyright 2013 James R. Garrison and Ryan V. Mishmash.
# Open source under the MIT license.  Source code at
# <https://github.com/simple-dmrg/simple-dmrg/>

# This code will run under any version of Python >= 2.6.  The following line
# provides consistency between python2 and python3.
from __future__ import print_function, division  # requires Python >= 2.6

# numpy and scipy imports
import numpy as np
from scipy.sparse import kron, identity
from scipy.sparse.linalg import eigsh  # Lanczos routine from ARPACK
from scipy.linalg import eigh
from scipy.sparse import csr_matrix

# We will use python's "namedtuple" to represent the Block and EnlargedBlock
# objects
from collections import namedtuple

In [2]:
Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])

def is_valid_block(block):
    for op in block.operator_dict.values():
        if op.shape[0] != block.basis_size or op.shape[1] != block.basis_size:
            return False
    return True

In [3]:
# This function should test the same exact things, so there is no need to
# repeat its definition.
is_valid_enlarged_block = is_valid_block

# Model-specific code for the Heisenberg XXZ chain
model_d = 3  # single-site basis size

Sz1 = csr_matrix([[1,0,0],[0,0,0],[0,0,-1]], dtype='d')
Sp1 = csr_matrix([[0,1,0],[0,0,1],[0,0,0]], dtype='d')

#H1 = np.array([[0, 0, 0], [0, 0, 0],[0, 0, 0]], dtype='d')  # single-site portion of H is zero
H1 = -10**(-4)*Sz1  # single-site portion of H is zero

In [4]:
def H2(Sz1, Sp1, Sz2, Sp2):  # two-site part of H
    """Given the operators S^z and S^+ on two sites in different Hilbert spaces
    (e.g. two blocks), returns a Kronecker product representing the
    corresponding two-site term in the Hamiltonian that joins the two sites.
    """
    J = 1.
    Jz = 1.
    return (
        (J / 2) * (kron(Sp1, csr_matrix.getH(Sp2)) + kron(csr_matrix.getH(Sp1), Sp2)) +
        Jz * kron(Sz1, Sz2)
    )

    # conn refers to the connection operator, that is, the operator on the edge of
    # the block, on the interior of the chain.  We need to be able to represent S^z
    # and S^+ on that site in the current basis in order to grow the chain.
initial_block = Block(length=1, basis_size=model_d, operator_dict={
    "H": H1,
    "conn_Sz": Sz1,
    "conn_Sp": Sp1,
})

In [5]:
def enlarge_block(block):
    """This function enlarges the provided Block by a single site, returning an
    EnlargedBlock.
    """
    mblock = block.basis_size
    o = block.operator_dict

    # Create the new operators for the enlarged block.  Our basis becomes a
    # Kronecker product of the Block basis and the single-site basis.  NOTE:
    # `kron` uses the tensor product convention making blocks of the second
    # array scaled by the first.  As such, we adopt this convention for
    # Kronecker products throughout the code.
    enlarged_operator_dict = {
        "H": kron(o["H"], identity(model_d)) + kron(identity(mblock), H1) + H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1),
        "conn_Sz": kron(identity(mblock), Sz1),
        "conn_Sp": kron(identity(mblock), Sp1),
    }

    return EnlargedBlock(length=(block.length + 1),
                         basis_size=(block.basis_size * model_d),
                         operator_dict=enlarged_operator_dict)

def rotate_and_truncate(operator, transformation_matrix):
    """Transforms the operator to the new (possibly truncated) basis given by
    `transformation_matrix`.
    """
    return csr_matrix.getH(transformation_matrix)@operator@transformation_matrix

In [6]:
def single_dmrg_step(sys, env, m):
    """Performs a single DMRG step using `sys` as the system and `env` as the
    environment, keeping a maximum of `m` states in the new basis.
    """
    assert is_valid_block(sys)
    assert is_valid_block(env)

    # Enlarge each block by a single site.
    sys_enl = enlarge_block(sys)
    if sys is env:  # no need to recalculate a second time
        env_enl = sys_enl
    else:
        env_enl = enlarge_block(env)

    assert is_valid_enlarged_block(sys_enl)
    assert is_valid_enlarged_block(env_enl)

    # Construct the full superblock Hamiltonian.
    m_sys_enl = sys_enl.basis_size
    m_env_enl = env_enl.basis_size
    sys_enl_op = sys_enl.operator_dict
    env_enl_op = env_enl.operator_dict
    superblock_hamiltonian = kron(sys_enl_op["H"], identity(m_env_enl)) + kron(identity(m_sys_enl), env_enl_op["H"]) + \
                             H2(sys_enl_op["conn_Sz"], sys_enl_op["conn_Sp"], env_enl_op["conn_Sz"], env_enl_op["conn_Sp"])

    # Call ARPACK to find the superblock ground state.  ("SA" means find the
    # "smallest in amplitude" eigenvalue.)
    (energy,), psi0 = eigsh(superblock_hamiltonian, k=1, which="SA")

    # Construct the reduced density matrix of the system by tracing out the
    # environment
    #
    # We want to make the (sys, env) indices correspond to (row, column) of a
    # matrix, respectively.  Since the environment (column) index updates most
    # quickly in our Kronecker product structure, psi0 is thus row-major ("C
    # style").
    psi0 = psi0.reshape([sys_enl.basis_size, -1], order="C")
    rho = csr_matrix(psi0)@csr_matrix.getH(psi0)

    # Diagonalize the reduced density matrix and sort the eigenvectors by
    # eigenvalue.
    evals, evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eval, evec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eval, evec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])  # largest eigenvalue first

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(len(possible_eigenstates), m)
    transformation_matrix = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
        transformation_matrix[:, i] = evec

    truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])
    print("truncation error:", truncation_error)

    # Rotate and truncate each operator.
    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    newblock = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)

    return newblock, energy, truncation_error, psi0, superblock_hamiltonian

In [8]:
L=10
m=300
block = initial_block
while 2 * block.length < L:
    print("L =", block.length * 2 + 2)
    block, energy, truncation_error, psi0, superblock_hamiltonian = single_dmrg_step(block, block, m=m)
    print("E/L =", energy / (block.length * 2))
    #print("truncation_error", truncation_error)

L = 4
truncation error: 0.0
E/L = -0.8453049750640259
L = 6
truncation error: -8.881784197001252e-16
E/L = -0.9208012317348633
L = 8
truncation error: -2.220446049250313e-15
E/L = -0.9615496472105414
L = 10
truncation error: 7.771561172376096e-16
E/L = -0.9863930552563671


In [18]:
kron(block.operator_dict['H'],block.operator_dict['H'])

C:\Users\Qiancan-Chen\anaconda3\lib\site-packages\scipy\sparse\bsr.py:299: RuntimeWarning: overflow encountered in int_scalars
  return int(self.indptr[-1] * R * C)


<59049x59049 sparse matrix of type '<class 'numpy.float64'>'
	with -808182895 stored elements (blocksize = 243x243) in Block Sparse Row format>

In [19]:
59049*59049

3486784401

In [10]:
psi0[0]

array([-2.39685609e-17,  1.99827140e-16,  3.54988304e-01, -4.33263217e-17,
        2.03928413e-16, -3.29223532e-01,  3.01766608e-17, -8.61767250e-02,
       -9.77489551e-17,  2.92109995e-17, -4.59308605e-06, -7.55664085e-16,
        1.01572133e-02,  4.81360124e-17,  3.77163409e-17,  2.16253864e-06,
        9.59734281e-18,  2.17934244e-16, -1.28999021e-17, -1.28286519e-16,
        6.39084344e-04,  7.12902881e-17,  1.12558985e-03, -1.90621166e-17,
       -2.51906945e-17,  3.09468474e-07, -1.37769838e-15,  7.80352331e-18,
       -9.10567403e-18, -5.67457114e-04,  2.00176929e-17,  5.61941536e-05,
       -8.79299240e-16, -2.76982570e-17, -3.82299404e-04, -6.46240737e-17,
        5.49105647e-16,  2.72845238e-05,  2.30281124e-13, -8.11243371e-18,
       -6.74299665e-07,  1.39223408e-15, -4.63790268e-16, -1.41296447e-13,
        5.06071333e-05, -1.91618292e-16, -3.93318489e-14, -2.27654927e-05,
       -7.66023744e-17, -2.58777691e-15, -6.71021186e-05, -2.71446915e-17,
       -5.54013154e-07,  

In [13]:
block.operator_dict['H']

array([[-4.41806075e+00,  1.14560902e-09,  1.79795386e-15, ...,
        -2.81336761e-06,  1.21288734e-06, -9.40453710e-06],
       [ 1.14560902e-09, -4.41786075e+00,  2.94481821e-16, ...,
        -1.31947440e-06,  2.90271304e-06, -1.03456227e-06],
       [ 1.79795386e-15,  2.94481821e-16, -3.59317355e+00, ...,
         2.45294374e-06,  1.19460472e-07, -4.81427444e-05],
       ...,
       [-2.81336761e-06, -1.31947440e-06,  2.45294374e-06, ...,
         1.01503190e+00, -8.45011488e-02, -4.12286312e-02],
       [ 1.21288734e-06,  2.90271304e-06,  1.19460472e-07, ...,
        -8.45011488e-02,  7.85475991e-01, -4.01300288e-04],
       [-9.40453710e-06, -1.03456227e-06, -4.81427444e-05, ...,
        -4.12286312e-02, -4.01300288e-04,  4.76530250e-01]])

In [ ]:
# Enlarge each block by a single site.
sys_enl = enlarge_block(block)
env_enl = sys_enl
env_enl = enlarge_block(block)


# Construct the full superblock Hamiltonian.
m_sys_enl = sys_enl.basis_size
m_env_enl = env_enl.basis_size
sys_enl_op = sys_enl.operator_dict
env_enl_op = env_enl.operator_dict
superblock_hamiltonian = kron(sys_enl_op["H"], identity(m_env_enl)) + kron(identity(m_sys_enl), env_enl_op["H"]) + \
                         H2(sys_enl_op["conn_Sz"], sys_enl_op["conn_Sp"], env_enl_op["conn_Sz"], env_enl_op["conn_Sp"])
kron(sys_enl_op["H"], identity(m_env_enl)) + kron(identity(m_sys_enl), env_enl_op["H"]) + \
                         H2(sys_enl_op["conn_Sz"], sys_enl_op["conn_Sp"], env_enl_op["conn_Sz"], env_enl_op["conn_Sp"])

In [15]:
kron(sys_enl_op["H"], identity(m_env_enl))

NameError: name 'sys_enl_op' is not defined

In [14]:
sys_enl_op["H"]

NameError: name 'sys_enl_op' is not defined

In [ ]:
identity(m_env_enl)

In [ ]:
243*243

In [ ]:
block.operator_dict['H']

In [ ]:
sparse.issparse(superblock_hamiltonian)

In [ ]:
6561*6561

In [ ]:
from scipy import sparse

In [ ]:
block.operator_dict['H'] = sparse.csr_matrix(block.operator_dict['H'])

In [ ]:
a =  np.array([7.771561172376096e-16],dtype='d')

In [17]:
kron(Sp1, csr_matrix.getH(Sp2))

NameError: name 'Sp2' is not defined